**Introduzione a LangChain e agli agenti RAG**

LangChain è una libreria Python progettata per facilitare l'integrazione dei Large Language Model (LLM) con dati esterni, strumenti e ambienti complessi. Il suo obiettivo principale è permettere lo sviluppo di applicazioni avanzate che non si limitano a generare testo, ma che possono interagire dinamicamente con fonti di conoscenza, API, database e sistemi esterni.

Tra le applicazioni più interessanti realizzabili con LangChain ci sono gli **agenti**, ovvero sistemi intelligenti che utilizzano un LLM per prendere decisioni, selezionare strumenti, pianificare azioni e risolvere compiti. Gli agenti sono fondamentali per automatizzare processi multi-step, dove serve non solo completare testi ma anche eseguire azioni guidate dal ragionamento.

Una delle architetture più efficaci per rispondere a domande specifiche su fonti informative complesse è quella degli **agenti RAG** (Retrieval-Augmented Generation). In questa struttura, l’agente combina due fasi principali:

1. **Retrieval** – Recupera documenti rilevanti da una base di conoscenza tramite tecniche di ricerca semantica o keyword-based.
2. **Generation** – Genera una risposta sintetica e coerente basandosi sui documenti recuperati.

Questo approccio permette di superare i limiti di memoria degli LLM, mantenendo la risposta aderente a una fonte verificabile. RAG è utilizzato in molte applicazioni reali, ad esempio per costruire chatbot aziendali, assistenti legali, motori di ricerca semantici e sistemi di supporto decisionale.

LangChain fornisce strumenti modulari per implementare agenti RAG, integrando moduli di embedding, vector store, chain di prompt, strumenti personalizzati e controllo del flusso logico.



---

### **Struttura di una tipica applicazione RAG**

Una tipica applicazione **Retrieval-Augmented Generation (RAG)** è composta da due componenti principali:

1. **Indicizzazione (Indexing)**
   Un processo che serve a caricare, suddividere e indicizzare i dati provenienti da una fonte. Questa fase viene eseguita **offline**, prima dell’interazione con l’utente.

2. **Recupero e generazione (Retrieval and Generation)**
   La fase eseguita **a runtime**, quando l’utente pone una domanda. Il sistema recupera i dati rilevanti dall’indice e li fornisce al modello, che genera la risposta.

> **Nota**: la fase di indicizzazione segue spesso lo stesso schema di un sistema di ricerca semantica.

---

### **Pipeline completa: dal dato grezzo alla risposta**

Il flusso completo più comune, dalla fonte di dati grezza alla risposta generata, prevede i seguenti passaggi:

#### **1. Indexing**

* **Load** – Caricamento dei dati
  I dati vengono caricati tramite **Document Loaders**, connettori che leggono file, pagine web, PDF, database, ecc.

* **Split** – Suddivisione in chunk
  I **Text Splitters** dividono i documenti in porzioni più piccole. Questo è utile sia per ottimizzare la ricerca, sia per assicurarsi che il contenuto rientri nel contesto gestibile del modello LLM (limite di token).

* **Store** – Salvataggio e indicizzazione
  Le porzioni di testo (chunk) vengono memorizzate e indicizzate all'interno di un **Vector Store**, dove ogni chunk è trasformato in un **embedding vettoriale**. Questo consente di effettuare ricerche semantiche rapide e precise.

---



![Alt text](rag_indexing-8160f90a90a33253d0154659cf7d453f.png)


---

### **2. Retrieval and Generation**

Una volta che i dati sono stati indicizzati, entra in gioco la seconda fase: **recupero e generazione**, eseguita ogni volta che l’utente pone una domanda.

#### **Retrieve – Recupero dei dati rilevanti**

A partire dall’input dell’utente (la **query**), il sistema recupera i chunk più pertinenti dalla memoria vettoriale.
Questo avviene tramite un componente chiamato **Retriever**, che esegue una ricerca semantica confrontando l’embedding della query con quelli memorizzati.

#### **Generate – Generazione della risposta**

I chunk recuperati, insieme alla domanda dell’utente, vengono inseriti in un **prompt** che viene poi fornito a un **ChatModel** o **LLM** (Large Language Model).
Il modello genera una risposta sfruttando il contesto fornito dal materiale recuperato, evitando allucinazioni e mantenendo l’aderenza ai dati reali.

---

Questa architettura consente di ottenere risposte più accurate e affidabili rispetto a un LLM standalone, specialmente quando è necessario rispondere su dati proprietari, documentazione interna o fonti aggiornabili.


![Alt text](rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)


---

### **Installazione**

Per seguire questo tutorial è necessario installare alcune dipendenze di **LangChain**.
Puoi farlo tramite **pip**.

#### **Installazione via pip**

```bash
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
```

Questo comando installa:

* `langchain-text-splitters`: per suddividere i documenti in chunk
* `langchain-community`: insieme di integrazioni e loader della community
* `langgraph`: framework per costruire agenti complessi tramite grafi


---


---

### **Componenti necessari**

Per costruire la nostra applicazione RAG, dobbiamo selezionare tre componenti fondamentali dalla suite di integrazioni offerte da LangChain:

1. **Modello di chat (LLM)**
2. **Modello di embeddings**
3. **Vector store (memoria vettoriale)**

---




### **1. Selezione del modello di chat (LLM)**

LangChain consente di utilizzare diversi provider per accedere a modelli di linguaggio come GPT-4, GPT-3.5, Claude, ecc.
In questo esempio, ci concentreremo su **modelli OpenAI compatibili**, scegliendo tra:

* **OpenAI** (API ufficiale)
* **Azure OpenAI**
* **LM Studio** (server locale compatibile con OpenAI)

Per tutti i casi, il modulo da utilizzare è `init_chat_model` di `langchain.chat_models`.

---

### **Installazione**

```bash
pip install -qU "langchain[openai]"
```

---

### **Opzione 1 – Connessione via OpenAI API (ufficiale)**

```python
import getpass
import os

# Imposta la chiave API se non già presente
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Inserisci la tua API Key OpenAI: ")

from langchain.chat_models import init_chat_model

# Inizializza un modello GPT-4o-mini usando OpenAI come provider
llm = init_chat_model("gpt-4o-mini", model_provider="openai")
```

---

### **Opzione 2 – Connessione via Azure OpenAI**

> Azure richiede di specificare **API key**, **endpoint** e **nome del deployment**.

```python
import getpass
import os

# Imposta le variabili per Azure OpenAI
os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Inserisci la Azure API Key: ")
os.environ["AZURE_OPENAI_ENDPOINT"] = input("Inserisci l'endpoint Azure (es. https://nome.cognitiveservices.azure.com): ")
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = input("Inserisci il nome del deployment (es. gpt-4o-deployment): ")

from langchain.chat_models import init_chat_model

llm = init_chat_model(
    "gpt-4o", 
    model_provider="azure",
)
```

LangChain rileva automaticamente le variabili `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT` e `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`.

---

### **Opzione 3 – Connessione via LM Studio (server locale compatibile OpenAI)**

> LM Studio espone un endpoint locale compatibile con OpenAI (tipicamente su `http://localhost:1234/v1`)

```python
import os
from langchain.chat_models import init_chat_model

# Configura l'accesso al server locale LM Studio
os.environ["OPENAI_API_KEY"] = "not-needed"
os.environ["OPENAI_BASE_URL"] = "http://localhost:1234/v1"

llm = init_chat_model(
    "lmstudio-model",           # Es: "mistral", "llama3", ecc.
    model_provider="openai"     # LM Studio è compatibile OpenAI, quindi usiamo questo
)
```

Puoi ottenere il nome esatto del modello aperto su LM Studio tramite l’interfaccia oppure controllando i log nel terminale.

---

### **Nota finale**

Il metodo `init_chat_model()` astrae la complessità del provider. Cambiare tra OpenAI, Azure o LM Studio è questione di cambiare:

* Il valore di `model_provider`
* Le variabili d’ambiente richieste




---

## **2. Selezione del modello di Embedding**

Gli **embedding** sono rappresentazioni numeriche (vettori) di frasi o documenti. Servono per misurare la **similarità semantica** tra testi e sono fondamentali per il recupero nella pipeline RAG.

LangChain consente l’uso di diversi modelli di embedding. In questa sezione vedremo:

* `text-embedding-3-large` (OpenAI – più recente e accurato)
* `text-embedding-ada-002` (OpenAI – più economico, legacy)
* `MiniLM` (open-source – via Hugging Face)

Ti mostrerò:

1. Come configurarli
2. Quando e perché usare ciascuno
3. Differenze chiave tra prestazioni, costi e compatibilità

---

###  **Installazione necessaria**

Per i modelli OpenAI:

```bash
pip install -qU langchain-openai
```

Per MiniLM (Hugging Face):

```bash
pip install -qU sentence-transformers langchain-community
```

---

## 🔹 **A. OpenAI – `text-embedding-3-large`**

> Ultimo modello embedding rilasciato da OpenAI. Supporta testo multilingua, alta accuratezza e compressione.

```python
import getpass
import os

# Imposta la chiave OpenAI se non presente
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Inserisci la tua OpenAI API Key: ")

from langchain_openai import OpenAIEmbeddings

# Modello embedding più accurato
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
```

**Vantaggi**:

* Alta precisione semantica
* Supporto multilingua
* Comprimibile a 1536 dimensioni
* Ottimo per applicazioni critiche (RAG, ricerca documentale)

**Svantaggi**:

* Costo più alto per token
* Richiede connessione alle API di OpenAI

---

## 🔹 **B. OpenAI – `text-embedding-ada-002`**

> Modello precedente, molto utilizzato per il buon rapporto qualità/prezzo.

```python
from langchain_openai import OpenAIEmbeddings

# Modello embedding economico e ancora valido
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
```

**Vantaggi**:

* Più economico del `3-large`
* Buona accuratezza per testi in inglese
* Bassa latenza

**Svantaggi**:

* Peggiore su testi multilingua
* Meno efficace su concetti complessi

**Quando usarlo**:

* Quando si lavora con grandi volumi di dati
* Quando il budget è limitato
* Quando si ha bisogno di embedding rapidi ed economici

---

## 🔹 **C. Hugging Face – `MiniLM` (open-source)**

> Alternativa gratuita, locale, utile per prototipazione e ambienti privati.

```python
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
```

**Vantaggi**:

* Gratuito
* Esecuzione locale (nessuna API esterna)
* Adatto a molti task generici di similarità semantica
* Più veloce dei modelli di grandi dimensioni

**Svantaggi**:

* Meno accurato di OpenAI 3-large
* Limitato su testi complessi o lunghi
* Solo supporto parziale multilingua

**Quando usarlo**:

* In progetti on-premise o air-gapped
* Per test e prototipi
* Quando non è disponibile una connessione a internet o si vuole evitare l’uso di servizi esterni

---

##  **Confronto sintetico**

| Modello                  | Precisione | Multilingua  | Latenza     | Costo    | Note                           |
| ------------------------ | ---------- | ------------ | ----------- | -------- | ------------------------------ |
| `text-embedding-3-large` | ⭐⭐⭐⭐⭐      | ✅            | Media       | Alta     | Miglior qualità generale       |
| `text-embedding-ada-002` | ⭐⭐⭐⭐       | ❌ (parziale) | Bassa       | Bassa    | Ottimo rapporto qualità/prezzo |
| `MiniLM` (`L6-v2`)       | ⭐⭐⭐        | ❌ (parziale) | Molto bassa | Gratuito | Ottimo per prototipi locali    |

---

##  **Conclusioni**

* Se vuoi **la massima qualità** per un'applicazione RAG in produzione → usa `text-embedding-3-large`.
* Se devi **ottimizzare i costi** su grandi dataset → considera `ada-002`.
* Se vuoi **lavorare offline o open-source** → MiniLM è la scelta giusta.

LangChain rende semplice cambiare embedding: basta modificare la riga di inizializzazione, mantenendo tutto il resto del sistema invariato.




### **Approfondimento: Cos'è un Embedding e perché è fondamentale nei sistemi RAG**

---

#### **1. Definizione di Embedding**

Un **embedding** è una rappresentazione numerica densa di un'informazione testuale (come una parola, una frase o un documento).
In pratica, trasforma il linguaggio naturale in **vettori di numeri** che catturano la **semantica** del testo, rendendo possibile confrontare, cercare e analizzare frasi con criteri matematici.

Esempio intuitivo:
La frase *"Come stai?"* e *"Tutto bene?"* avranno embedding simili, perché esprimono un significato vicino.
Al contrario, *"Apri la porta"* e *"Mangia una mela"* avranno embedding distanti.

---

#### **2. Perché gli Embedding sono centrali nei sistemi RAG**

RAG significa **Retrieval-Augmented Generation**. È un'architettura che combina:

* **retrieval** (recupero di contenuti rilevanti)
* **generation** (generazione di risposte da parte di un LLM)

Il recupero dei contenuti si basa **quasi sempre su embedding**, ed è qui che il loro ruolo diventa cruciale.

**Come funziona:**

1. Tutti i documenti vengono suddivisi in "chunk" (pezzi di testo) → *text splitting*
2. Ogni chunk viene trasformato in un **vettore embedding**
3. Quando l’utente fa una domanda:

   * La domanda viene **anch’essa convertita in embedding**
   * Viene calcolata la **distanza vettoriale** tra l’embedding della domanda e quelli dei documenti
   * I chunk più “vicini” vengono recuperati e forniti al modello generativo

> Senza embedding, non potremmo confrontare in modo efficiente frasi in linguaggio naturale per similarità di significato.

---

#### **3. Proprietà di un buon embedding**

Un buon modello di embedding deve:

* Catturare **relazioni semantiche**, non solo sintattiche
* Supportare **testi multilingua** se necessario
* Essere **compatto ma informativo** (vettori densi, es. 384–1536 dimensioni)
* Essere **veloce da calcolare** anche su grandi quantità di testo

---

#### **4. Tipologie di modelli di embedding**

| Tipo                           | Esempi                                                    | Caratteristiche principali                                |
| ------------------------------ | --------------------------------------------------------- | --------------------------------------------------------- |
| **Pre-addestrati commerciali** | OpenAI `text-embedding-ada-002`, `text-embedding-3-large` | Alta qualità, multilingua, ma richiedono API a pagamento  |
| **Modelli open-source**        | MiniLM, BGE, Instructor, GTE                              | Gratuiti, eseguibili in locale, meno accurati in generale |
| **Custom**                     | Addestrati su dati aziendali                              | Ottimizzati per il dominio specifico                      |

---

#### **5. Distanza vettoriale e similarità**

Gli embedding non servono da soli: vanno confrontati con una **metrica di similarità**. Le più comuni sono:

* **Cosine similarity** – misura l’angolo tra vettori (indipendente dalla lunghezza)
* **Distanza L2 (euclidea)** – usata ad esempio da FAISS `IndexFlatL2`
* **Dot product** – usata spesso in modelli neurali

Queste metriche permettono di recuperare i chunk “più simili” alla query in modo efficiente, anche su milioni di documenti.

---

#### **6. Prestazioni: qualità vs velocità**

Scegliere il giusto embedding dipende dal contesto:

| Contesto                                       | Modello consigliato      |
| ---------------------------------------------- | ------------------------ |
| RAG aziendale di qualità su dati in più lingue | `text-embedding-3-large` |
| Ricerca veloce su documenti in inglese         | `text-embedding-ada-002` |
| Sistema locale, senza costi o cloud            | `MiniLM`, `BGE`, `GTE`   |

---

#### **7. Visualizzazione (opzionale)**

È possibile proiettare gli embedding in 2D o 3D (es. con PCA o UMAP) per visualizzare la distribuzione dei significati nel testo.
Questa analisi è utile per verificare se frasi simili vengono effettivamente mappate vicino tra loro.

---

#### **8. Rischi e best practice**

* **Chunk troppo lunghi** portano a embedding poco precisi → usa text splitter
* **Contenuto rumoroso** o ripetitivo può alterare i risultati → filtra o pulisci prima
* **Usa batching** per calcolare più embedding in parallelo e risparmiare tempo

---

### **Conclusione**

Gli embedding sono la base invisibile ma fondamentale di qualsiasi sistema di retrieval semantico.
Che si tratti di un motore RAG, un motore di ricerca AI, o un sistema di classificazione testi, il **modo in cui trasformi il testo in numeri** determina tutta la qualità della tua pipeline.



---

## **3. Selezione del Vector Store**

Il **vector store** è il componente responsabile dell’indicizzazione, memorizzazione e ricerca dei **vettori embedding**.
È parte fondamentale in ogni sistema **RAG**, perché permette di **recuperare documenti simili semanticamente** a partire dalla query dell’utente.

LangChain supporta numerosi vector store, tra cui:

* **FAISS** – locale, veloce, open-source
* **Qdrant** – scalabile, open-source, API-based o locale
* **Pinecone** – cloud-native, ottimizzato per applicazioni su larga scala

---

###  **Installazione base**

```bash
pip install -qU langchain-community
```

---

## 🔹 **Esempio 1 – FAISS (locale, veloce, open-source)**

**FAISS** è una libreria sviluppata da Facebook AI Research. Ottima per test locali, prototipi e ambienti controllati.

### Codice:

```python
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# Calcola la dimensione degli embedding
embedding_dim = len(embeddings.embed_query("hello world"))

# Crea un indice FAISS con distanza L2
index = faiss.IndexFlatL2(embedding_dim)

# Inizializza il vector store
vs = FAISS.from_documents(
        documents=chunks,
        embedding=embeddings
)
```

### Quando usarlo:

* Ambienti locali
* Nessuna dipendenza da servizi esterni
* Dataset medio-piccoli

---

## 🔹 **Esempio 2 – Qdrant (cloud o locale, open-source)**

**Qdrant** è un motore vettoriale moderno scritto in Rust, molto efficiente, scalabile e con supporto per **payload metadata**, **filtri**, **search ibrida**.

### Installazione:

```bash
pip install qdrant-client langchain-community
```

### Codice (con server Qdrant locale su `http://localhost:6333`):

```python
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from langchain.schema import Document

# Connessione a Qdrant
client = QdrantClient(host="localhost", port=6333)

# Inizializzazione dello store vettoriale
vector_store = Qdrant.from_documents(
    documents=[Document(page_content="Test", metadata={"source": "example"})],
    embedding=embeddings,
    location="http://localhost:6333",
    collection_name="rag_example",
    client=client,
)
```

### Quando usarlo:

* Hai bisogno di metadati avanzati e filtri
* Deployment on-premise o su cloud
* Performance e scalabilità superiori a FAISS

---

## 🔹 **Esempio 3 – Pinecone (cloud-native, altamente scalabile)**

**Pinecone** è un vector database professionale gestito, ottimo per applicazioni in produzione con milioni di documenti. Offre **alta disponibilità, replica, filtraggio, gestione di namespace**, ecc.

### Installazione:

```bash
pip install pinecone-client langchain-community
```

### Codice:

```python
import pinecone
from langchain_community.vectorstores import Pinecone
from langchain.schema import Document

# Inizializza Pinecone con API key e ambiente
pinecone.init(
    api_key="YOUR_PINECONE_API_KEY",
    environment="us-east1-gcp"  # esempio, dipende dal tuo account
)

# Crea l'indice se non esiste
index_name = "rag-demo"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=len(embeddings.embed_query("test")))

# Connessione all’indice
index = pinecone.Index(index_name)

# Crea il vector store
vector_store = Pinecone.from_documents(
    documents=[Document(page_content="Questo è un documento", metadata={"source": "A"})],
    embedding=embeddings,
    index_name=index_name,
)
```

### Quando usarlo:

* Applicazioni enterprise
* Dataset molto grandi
* Hai bisogno di disponibilità e replicazione automatica

---

##  **Confronto tra FAISS, Qdrant e Pinecone**

| Caratteristica    | FAISS             | Qdrant                  | Pinecone              |
| ----------------- | ----------------- | ----------------------- | --------------------- |
| Tipo              | Locale            | Cloud/Locale            | Solo Cloud            |
| Performance       | Alta (locale)     | Alta (cloud & locale)   | Altissima (cloud)     |
| Filtri avanzati   | ❌                 | ✅                       | ✅                     |
| Supporto metadati | ❌ (limitato)      | ✅                       | ✅                     |
| Persistenza       | Manuale           | Integrata               | Integrata             |
| Costo             | Gratuito (locale) | Gratuito / self-hosting | A pagamento           |
| Casistica ideale  | Prototipi         | Produzione flessibile   | Produzione enterprise |

---

###  **Conclusione**

La scelta del vector store dipende dal contesto d’uso:

* **FAISS**: perfetto per test, prototipi, sviluppo locale
* **Qdrant**: ideale se vuoi scalabilità, open-source e controllo fine
* **Pinecone**: soluzione cloud altamente affidabile per produzione su larga scala

Tutti i vector store funzionano allo stesso modo: ricevono i vettori dagli **embedding** e li rendono ricercabili per similarità.
LangChain rende il passaggio da uno all’altro molto semplice, mantenendo la stessa interfaccia di utilizzo per l’intero flusso RAG.

## ATTENZIONE
Faiss nella sua versione più recente non ha bisogno dell'indice esterni


---

## Approfondimento: utilizzo di FAISS come Vector Store in LangChain

### Introduzione

FAISS (Facebook AI Similarity Search) è una libreria open-source sviluppata da Meta per effettuare ricerche vettoriali efficienti su larga scala. Viene utilizzata per confrontare rappresentazioni numeriche (embedding) di documenti al fine di recuperarne quelli semanticamente più simili a una query.

In LangChain, FAISS è uno dei vector store supportati per costruire pipeline di retrieval all'interno di applicazioni RAG (Retrieval-Augmented Generation). Di seguito viene mostrato un esempio pratico completo e una spiegazione di tutti i parametri rilevanti.

---

### Installazione

Per utilizzare FAISS con LangChain e modelli Hugging Face:

```bash
pip install faiss-cpu langchain-community sentence-transformers
```

---

### Inizializzazione: embedding Hugging Face e documenti di esempio

```python
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# Inizializza un modello di embedding gratuito e locale
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Documenti simulati in inglese
documents = [
    Document(page_content="LangChain is a powerful framework for building LLM applications."),
    Document(page_content="FAISS enables fast semantic search over text data."),
    Document(page_content="Python is widely used in AI and machine learning projects."),
    Document(page_content="Vector databases store dense representations of text."),
]
```

---

### Costruzione dell’indice FAISS

```python
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# Calcola la dimensione dei vettori di embedding
embedding_dim = len(embeddings.embed_query("test"))

def build_faiss_vectorstore(chunks: List[Document], embeddings: HuggingFaceEmbeddings, persist_dir: str) -> FAISS:
    """
    Costruisce da zero un FAISS index (IndexFlatL2) e lo salva su disco.
    """
    # Determina la dimensione dell'embedding
    vs = FAISS.from_documents(
        documents=chunks,
        embedding=embeddings
    )

    Path(persist_dir).mkdir(parents=True, exist_ok=True)
    vs.save_local(persist_dir)
    return vs

```


---

### Ricerca semantica

È possibile effettuare una ricerca semantica a partire da una query in linguaggio naturale:

```python
results = vector_store.similarity_search("How can I build AI applications?", k=2)
for result in results:
    print(result.page_content)
```

LangChain calcola l'embedding della query, esegue la ricerca nel vector store e restituisce i documenti più vicini semanticamente.

---

### Salvataggio dell’indice FAISS su disco

Per evitare di ricostruire l’indice ogni volta, è possibile salvarlo localmente:

```python
vector_store.save_local("faiss_index_example")
```

Questo salverà due file nella directory `faiss_index_example/`:

* `index.faiss`: rappresentazione binaria dell’indice vettoriale
* `index.pkl`: metadati, mappature e documenti

---

### Caricamento dell’indice FAISS già costruito

In un secondo momento, è possibile ricaricare l’indice e riprendere l’uso del vector store:

```python
vector_store = FAISS.load_local("faiss_index_example", embeddings)
```

In questo modo si evita il costo computazionale della rigenerazione degli embedding e dell’indice.

---

### Uso come retriever in una pipeline RAG

LangChain consente di convertire il vector store in un retriever standard compatibile con le catene di domanda-risposta:

```python
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
```

Questo retriever può essere usato come componente all’interno di una catena `RetrievalQA` o `ConversationalRetrievalChain`.

---

### Tipologie di indice FAISS

| Tipo FAISS      | Descrizione                                                              |
| --------------- | ------------------------------------------------------------------------ |
| `IndexFlatL2`   | Ricerca esatta con distanza euclidea                                     |
| `IndexFlatIP`   | Ricerca esatta con prodotto scalare (dot product)                        |
| `IndexIVFFlat`  | Ricerca approssimata basata su clustering (richiede addizionale `train`) |
| `IndexHNSWFlat` | Ricerca approssimata tramite grafo navigabile                            |

Per progetti semplici o prototipi, `IndexFlatL2` è sufficiente e non richiede training. Per dataset molto grandi, è possibile utilizzare indici approssimati come `IndexIVFFlat` o `HNSW`.

---

### Considerazioni finali

FAISS è una soluzione locale ad alte prestazioni per la ricerca semantica su documenti testuali. In combinazione con LangChain e modelli di embedding gratuiti come MiniLM, consente di costruire pipeline RAG complete senza dover dipendere da API esterne.

Per applicazioni scalabili e in produzione, può essere utile valutare alternative cloud come Qdrant o Pinecone, ma FAISS resta un’ottima scelta per prototipi, ambienti controllati, e applicazioni offline.


## Embedding Dim
```python
embedding_dim = len(embeddings.embed_query("test"))
```

Nel contesto di LangChain e degli embedding, questa istruzione ha una funzione fondamentale: **determina la dimensione (numero di componenti) del vettore embedding prodotto dal modello**. Vediamo perché è importante e come funziona sotto il cofano.

---

### Cosa fa questa istruzione?

* `embeddings.embed_query("test")` restituisce un vettore embedding (una lista di numeri in virgola mobile) per la stringa `"test"`.
* `len(...)` calcola la **lunghezza di quella lista**, ovvero il numero di dimensioni (componenti) del vettore embedding.

Questa dimensione è essenziale per inizializzare un indice FAISS correttamente, perché FAISS richiede di sapere quanti elementi conterrà ogni vettore per costruire l’indice.

---

### Perché non esiste un metodo esplicito per ottenerla?

LangChain non espone direttamente la dimensione degli embedding generati. In particolare, per l’implementazione `HuggingFaceEmbeddings`, non c’è un attributo come `embeddings.dimension`, motivo per cui molti sviluppatori utilizzano la soluzione più pratica:

> Chiedere direttamente al modello: generare un embedding su una query di prova e misurarne la lunghezza.
> ([Stack Overflow][1])

---

### Dimensioni comuni di embedding Hugging Face

Un esempio frequente è il modello `all-MiniLM-L6-v2` di Sentence Transformers. Questo modello mappa ogni frase in uno spazio vettoriale di **384 dimensioni**.
([Hugging Face][2])

Ciò significa che l’istruzione

```python
len(embeddings.embed_query("test"))
```

restituirà 384 quando `embeddings` è basato su `all-MiniLM-L6-v2`.

---

### Scenario pratico con FAISS

Ecco il flusso completo per utilizzare questa informazione nel creare un indice FAISS:

```python
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
from langchain.schema import Document

# 1. Embedding open-source gratuito
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2. Documenti di esempio
documents = [
    Document(page_content="LangChain is great for building intelligent systems."),
    Document(page_content="FAISS enables quick semantic search."),
]

# 3. Calcola la dimensione del vettore embedding
embedding_dim = len(embeddings.embed_query("test"))

# 4. Costruisci un indice FAISS basato su distanza euclidea (L2)
index = faiss.IndexFlatL2(embedding_dim)

# 5. Costruisci il vector store
vector_store = FAISS.from_documents(
    documents=documents,
    embedding=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)
```

Così il vettore fornito da `embed_query("test")` garantisce che `IndexFlatL2(embedding_dim)` sia inizializzato con la dimensione corretta, evitando errori e garantendo integrità nel salvataggio e ricerca dell’indice.

---

### In sintesi

* **Scopo**: Determinare il numero di dimensioni del vettore embedding generato dal modello.
* **Motivo**: LangChain attualmente non esprime questa informazione direttamente, quindi si utilizza un embedding di prova come workaround. 
* **Esempio pratico**: `all-MiniLM-L6-v2` restituisce embedding da 384 componenti. 
* **Applicazione**: Questa dimensione è fondamentale per inizializzare correttamente un indice Faiss, garantendo coerenza nel confronto vettoriale.




---

## Similarity Metrics: Cosine, Dot Product, L2

Quando si confrontano vettori di embedding, è necessario misurare **quanto sono simili due vettori nello spazio vettoriale**. Le metriche di similarità sono funzioni matematiche che restituiscono una misura numerica di quanto due vettori siano “vicini” tra loro.

Le metriche più usate nei sistemi di ricerca semantica sono:

* **Cosine Similarity**
* **Dot Product (Inner Product)**
* **L2 Distance (Euclidean Distance)**

Ognuna ha caratteristiche, vantaggi e casi d'uso specifici.

---

### 1. Cosine Similarity

#### Definizione

La **cosine similarity** misura il **coseno dell’angolo** tra due vettori. Più piccolo è l’angolo, più simili sono i vettori.

$$
\text{cosine\_similarity}(A, B) = \frac{A \cdot B}{\|A\| \cdot \|B\|}
$$

Dove:

* $A \cdot B$ è il prodotto scalare (dot product)
* $\|A\|$ e $\|B\|$ sono le norme (lunghezze) dei vettori

#### Valori

* $1$: perfettamente simili (stesso verso)
* $0$: ortogonali (nessuna similarità)
* $-1$: direzioni opposte

#### Pro e contro

| Vantaggi                           | Svantaggi                |
| ---------------------------------- | ------------------------ |
| Invariante rispetto alla lunghezza | Richiede normalizzazione |
| Ottimo per misurare orientamento   | Non cattura magnitudine  |

#### Quando usarla

* Quando ti interessa **la direzione del significato** piuttosto che la sua intensità
* Per testi di **lunghezza variabile** e **embedding normalizzati**

---

### 2. Dot Product (Inner Product)

#### Definizione

Il **dot product** è una versione semplificata del cosine similarity **senza normalizzazione**. Misura l’allineamento e la magnitudine dei vettori:

$$
\text{dot\_product}(A, B) = \sum_i A_i \cdot B_i
$$

#### Valori

* Più alto è il valore, più simili sono i vettori

#### Pro e contro

| Vantaggi                                        | Svantaggi                            |
| ----------------------------------------------- | ------------------------------------ |
| Veloce da calcolare                             | Sensibile alla lunghezza dei vettori |
| Supportato nativamente da FAISS (`IndexFlatIP`) | Può penalizzare vettori corti        |

#### Quando usarla

* Quando gli **embedding non sono normalizzati**
* Quando usi modelli dove la magnitudine del vettore ha significato (es. classificazione o ranking)
* In contesti **dove vuoi priorità su "importanza" oltre alla direzione**

---

### 3. L2 Distance (Euclidean Distance)

#### Definizione

La **L2 distance** è la distanza euclidea standard tra due vettori:

$$
\text{L2}(A, B) = \sqrt{\sum_i (A_i - B_i)^2}
$$

Più piccolo è il valore, **più simili** sono i vettori.

#### Pro e contro

| Vantaggi                                        | Svantaggi                    |
| ----------------------------------------------- | ---------------------------- |
| Intuitiva e diretta                             | Sensibile alla scala         |
| Supportata nativamente in FAISS (`IndexFlatL2`) | Necessita embedding uniformi |

#### Quando usarla

* Quando vuoi una metrica geometrica diretta
* In applicazioni con embedding generati in ambienti controllati e normalizzati
* Quando **non normalizzi** i vettori e vuoi valutare distanza “fisica”

---

## Riepilogo comparativo

| Metrica           | Range tipico          | Richiede normalizzazione | Tipico uso in FAISS      | Adatta per                |
| ----------------- | --------------------- | ------------------------ | ------------------------ | ------------------------- |
| Cosine Similarity | -1 a 1                | Sì                       | Con `IndexFlatIP + norm` | Similarità semantica pura |
| Dot Product       | $-\infty$ a $+\infty$ | No                       | `IndexFlatIP`            | Sistemi di ranking        |
| L2 Distance       | $[0, +\infty)$        | No                       | `IndexFlatL2`            | Misura geometrica         |

---

## Quale metrica scegliere?

### Usa Cosine Similarity se:

* Gli embedding sono normalizzati
* Vuoi solo misurare **orientamento semantico**
* Usi modelli come `sentence-transformers` con `cosine similarity` come obiettivo

### Usa Dot Product se:

* L’ampiezza del vettore embedding **ha significato**
* Usi FAISS con `IndexFlatIP` e non vuoi normalizzare

### Usa L2 Distance se:

* Vuoi una **metrica geometrica diretta**
* Usi `IndexFlatL2` in FAISS
* Gli embedding non sono normalizzati ma comparabili

---

## Esempio pratico con FAISS in LangChain

### Cosine similarity con vettori normalizzati

```python
import numpy as np
import faiss

# Normalizza i vettori
def normalize(vectors):
    return vectors / np.linalg.norm(vectors, axis=1, keepdims=True)

# Usiamo IndexFlatIP ma normalizzando prima
vectors = np.array([[1.0, 2.0], [2.0, 3.0]], dtype="float32")
vectors = normalize(vectors)

index = faiss.IndexFlatIP(2)
index.add(vectors)

# Anche la query va normalizzata
query = normalize(np.array([[1.0, 1.5]], dtype="float32"))
D, I = index.search(query, k=1)
```

---

### L2 distance (ricerca esatta, non normalizzata)

```python
index = faiss.IndexFlatL2(2)
index.add(np.array([[1.0, 2.0], [2.0, 3.0]], dtype="float32"))
query = np.array([[1.0, 1.5]], dtype="float32")
D, I = index.search(query, k=1)
```

---

## Conclusione

La metrica di similarità è un **elemento critico in un sistema RAG o di ricerca semantica**, perché influenza direttamente i risultati restituiti. Scegliere la metrica giusta dipende:

* dal tipo di modello di embedding utilizzato
* da come gli embedding sono normalizzati o strutturati
* dal comportamento desiderato (priorità alla direzione semantica, magnitudine, o distanza)

LangChain e FAISS forniscono gli strumenti per lavorare con tutte queste metriche in modo flessibile e integrato.



---

## **Hybrid Search: BM25 + Vector Re-ranking**

### **Cos'è la Hybrid Search?**

La **Hybrid Search** (ricerca ibrida) è una tecnica che **combina due approcci diversi** per migliorare la qualità del recupero dei documenti in risposta a una query:

1. **BM25** (ricerca testuale classica, keyword-based)
2. **Vector Search** (ricerca semantica tramite embedding)

L’obiettivo è **sfruttare i punti di forza di entrambi**:

| Approccio         | Vantaggi principali                           | Limiti principali                                             |
| ----------------- | --------------------------------------------- | ------------------------------------------------------------- |
| **BM25**          | Ottimo per match di parole esatte, efficiente | Non comprende il significato semantico                        |
| **Vector Search** | Capisce sinonimi e concetti simili            | Può restituire risultati semanticamente vicini ma irrilevanti |

La ricerca ibrida consente di **aumentare precisione e recall**, specialmente in contesti dove è importante sia la **pertinenza semantica**, sia il **match testuale**.

---

### **Come funziona una pipeline BM25 + Vector Re-ranking**

1. **BM25** recupera un set iniziale di documenti **basato su corrispondenze di parole chiave**.
2. Questi documenti vengono **embeddingizzati**.
3. La query viene trasformata in un embedding vettoriale.
4. Si calcola la **similarità vettoriale** tra la query e ciascun documento del set iniziale.
5. I risultati vengono **riordinati (re-ranked)** in base alla similarità semantica.

In alternativa:

* si può fare **merge score-based**: combinando punteggi BM25 e vettoriali con pesi ($\alpha \cdot \text{bm25\_score} + (1 - \alpha) \cdot \text{vector\_similarity}$)

---

## **Cos'è Qdrant e perché è adatto alla ricerca ibrida**

**Qdrant** è un **motore di ricerca vettoriale open-source**, moderno, scritto in Rust, pensato per la ricerca scalabile e semantica.
A differenza di FAISS, Qdrant supporta **filtri**, **payload JSON**, **metadati** e **ricerca ibrida** nativamente.

### Caratteristiche principali di Qdrant:

* Supporto nativo per **BM25**
* Supporto nativo per **Vector search**
* Supporto per **filtri combinati**, **payload personalizzati**, **tag**
* API REST o gRPC + supporto per client Python (`qdrant-client`)
* Persistenza automatica e clustering

---

## **Installazione**

### Server Qdrant (opzioni):

* Via Docker:

```bash
docker run -p 6333:6333 -p 6334:6334 qdrant/qdrant
```

### Python client:

```bash
pip install qdrant-client langchain-community
```

---

## **Esempio pratico: Hybrid Search con Qdrant**

### 1. Inizializzazione embedding e client

```python
from qdrant_client import QdrantClient
from langchain_community.embeddings import HuggingFaceEmbeddings

# Embedding model open-source
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Connessione al server Qdrant locale
client = QdrantClient(host="localhost", port=6333)
```

---

### 2. Creazione della collection (con supporto hybrid)

```python
# Dimensione dell'embedding
dim = len(embedding_model.embed_query("example"))

client.recreate_collection(
    collection_name="hybrid_demo",
    vectors_config={"default": {"size": dim, "distance": "Cosine"}},
    optimizers_config={"default_segment_number": 1},
)
```

---

### 3. Inserimento dei documenti

```python
from langchain.schema import Document

documents = [
    Document(page_content="LangChain is a powerful framework for building LLM applications.", metadata={"id": "doc1"}),
    Document(page_content="FAISS enables fast semantic search over text data.", metadata={"id": "doc2"}),
    Document(page_content="Python is widely used in AI projects.", metadata={"id": "doc3"}),
]

from langchain_community.vectorstores import Qdrant

vector_store = Qdrant.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="hybrid_demo",
    client=client
)
```

---

### 4. Esecuzione di una Hybrid Search

```python
query = "How can I build AI applications?"

# Metodo 1: solo vector search
results_vector = vector_store.similarity_search(query, k=2)

# Metodo 2: hybrid search con BM25 + vector
results_hybrid = client.search(
    collection_name="hybrid_demo",
    query_vector=embedding_model.embed_query(query),
    with_payload=True,
    limit=3,
    with_vectors=False,
    score_threshold=None,
    params={
        "exact": False,
        "hybrid": {
            "alpha": 0.5,       # bilanciamento: 0 = solo BM25, 1 = solo vettore
            "vector": True,
        }
    }
)

for hit in results_hybrid:
    print(hit.payload)
```

---

## **Vantaggi della ricerca ibrida con Qdrant**

1. **Precisione migliorata** – Recupera documenti con match testuale *e* significato simile.
2. **Supporto nativo** – Nessuna logica custom: BM25 + embedding sono integrati.
3. **Filtri avanzati** – Puoi filtrare per metadati (es. categoria, data, autore).
4. **Persistenza automatica** – Non c'è bisogno di gestire manualmente salvataggi.

---

## **Quando usare la Hybrid Search**

* Quando i documenti contengono **termini tecnici precisi** ma anche sinonimi e varianti linguistiche.
* Quando vuoi **evitare risultati semanticamente simili ma fuori tema**.
* In domini come **ambiti legali, medici, documentazione tecnica**, dove sia il significato che la terminologia contano.

---

## **Conclusione**

La Hybrid Search combina il meglio dei due mondi: la precisione delle keyword (BM25) e la flessibilità semantica degli embedding.
**Qdrant** è uno dei pochi vector store a supportare questa modalità **nativamente**, rendendolo una scelta eccellente per applicazioni RAG moderne, scalabili e sensibili al contesto.


## progetto RAG completo

* **Documenti simulati** (in inglese)
* **FAISS** come vector store (senza hybrid search)
* **Embedding open-source gratuito** di Hugging Face (`all-MiniLM-L6-v2`)
* **LM Studio** come LLM per rispondere alle domande (server OpenAI-compatible)

Il codice include le ottimizzazioni: text splitting accurato, FAISS persistente (save/load per evitare rebuild), retriever con **MMR** per diversificazione dei risultati, prompt con **citazioni** alle fonti e struttura modulare “production-ready”.

---

## Requisiti

```bash
pip install -qU faiss-cpu langchain langchain-community sentence-transformers
```

> Assicurati che **LM Studio** sia in esecuzione e stia esponendo un endpoint OpenAI-compatible (default: `http://localhost:1234/v1`).
> Carica un modello in LM Studio (es. Mistral, Llama) e annotane il nome visualizzato.

---

## Variabili d’ambiente (LM Studio)

Nel tuo terminale (oppure impostale a runtime nel codice):

```bash
# LM Studio espone un endpoint OpenAI-compatible
export OPENAI_BASE_URL="http://localhost:1234/v1"
export OPENAI_API_KEY="not-needed"   # richiesto dall'SDK ma non usato
export LMSTUDIO_MODEL="mistral"      # usa il nome del modello caricato in LM Studio
```

Su Windows (PowerShell):

```powershell
$env:OPENAI_BASE_URL="http://localhost:1234/v1"
$env:OPENAI_API_KEY="not-needed"
$env:LMSTUDIO_MODEL="mistral"
```

---

## Codice completo

> Salvalo come `rag_faiss_lmstudio.py` e avvialo.

```python
from __future__ import annotations

import os
from dataclasses import dataclass
from pathlib import Path
from typing import List

import faiss
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.text_splitter import RecursiveCharacterTextSplitter

# LangChain Core (prompt/chain)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Chat model init (provider-agnostic, qui puntiamo a LM Studio via OpenAI-compatible)
from langchain.chat_models import init_chat_model


# =========================
# Configurazione
# =========================

@dataclass
class Settings:
    # Persistenza FAISS
    persist_dir: str = "faiss_index_example"
    # Text splitting
    chunk_size: int = 700
    chunk_overlap: int = 100
    # Retriever (MMR)
    search_type: str = "mmr"        # "mmr" o "similarity"
    k: int = 4                      # risultati finali
    fetch_k: int = 20               # candidati iniziali (per MMR)
    mmr_lambda: float = 0.3         # 0 = diversificazione massima, 1 = pertinenza massima
    # Embedding
    hf_model_name: str = "sentence-transformers/all-MiniLM-L6-v2"
    # LM Studio (OpenAI-compatible)
    lmstudio_model_env: str = "LMSTUDIO_MODEL"  # nome del modello in LM Studio, via env var


SETTINGS = Settings()


# =========================
# Componenti di base
# =========================

def get_embeddings(settings: Settings) -> HuggingFaceEmbeddings:
    """
    Restituisce un modello di embedding locale e gratuito (Hugging Face).
    """
    return HuggingFaceEmbeddings(model_name=settings.hf_model_name)


def get_llm_from_lmstudio(settings: Settings):
    """
    Inizializza un ChatModel puntando a LM Studio (OpenAI-compatible).
    Richiede:
      - OPENAI_BASE_URL (es. http://localhost:1234/v1)
      - OPENAI_API_KEY (placeholder qualsiasi, es. "not-needed")
      - LMSTUDIO_MODEL (nome del modello caricato in LM Studio)
    """
    base_url = os.getenv("OPENAI_BASE_URL")
    api_key = os.getenv("OPENAI_API_KEY")
    model_name = os.getenv(settings.lmstudio_model_env)

    if not base_url or not api_key:
        raise RuntimeError(
            "OPENAI_BASE_URL e OPENAI_API_KEY devono essere impostate per LM Studio."
        )
    if not model_name:
        raise RuntimeError(
            f"Imposta la variabile {settings.lmstudio_model_env} con il nome del modello caricato in LM Studio."
        )

    # model_provider="openai" perché l'endpoint è OpenAI-compatible
    return init_chat_model(model_name, model_provider="openai")


def simulate_corpus() -> List[Document]:
    """
    Crea un piccolo corpus di documenti in inglese con metadati e 'source' per citazioni.
    """
    docs = [
        Document(
            page_content=(
                "LangChain is a framework that helps developers build applications "
                "powered by Large Language Models (LLMs). It provides chains, agents, "
                "prompt templates, memory, and integrations with vector stores."
            ),
            metadata={"id": "doc1", "source": "intro-langchain.md"}
        ),
        Document(
            page_content=(
                "FAISS is a library for efficient similarity search and clustering of dense vectors. "
                "It supports exact and approximate nearest neighbor search and scales to millions of vectors."
            ),
            metadata={"id": "doc2", "source": "faiss-overview.md"}
        ),
        Document(
            page_content=(
                "Sentence-transformers like all-MiniLM-L6-v2 produce sentence embeddings suitable "
                "for semantic search, clustering, and information retrieval. The embedding size is 384."
            ),
            metadata={"id": "doc3", "source": "embeddings-minilm.md"}
        ),
        Document(
            page_content=(
                "A typical RAG pipeline includes indexing (load, split, embed, store) and "
                "retrieval+generation. Retrieval selects the most relevant chunks, and the LLM produces "
                "an answer grounded in those chunks."
            ),
            metadata={"id": "doc4", "source": "rag-pipeline.md"}
        ),
        Document(
            page_content=(
                "Maximal Marginal Relevance (MMR) balances relevance and diversity during retrieval. "
                "It helps avoid redundant chunks and improves coverage of different aspects."
            ),
            metadata={"id": "doc5", "source": "retrieval-mmr.md"}
        ),
    ]
    return docs


def split_documents(docs: List[Document], settings: Settings) -> List[Document]:
    """
    Applica uno splitting robusto ai documenti per ottimizzare il retrieval.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=settings.chunk_size,
        chunk_overlap=settings.chunk_overlap,
        separators=[
            "\n\n", "\n", ". ", "? ", "! ", "; ", ": ",
            ", ", " ", ""  # fallback aggressivo
        ],
    )
    return splitter.split_documents(docs)


def build_faiss_vectorstore(chunks: List[Document], embeddings: HuggingFaceEmbeddings, persist_dir: str) -> FAISS:
    """
    Costruisce da zero un FAISS index (IndexFlatL2) e lo salva su disco.
    """
    # Determina la dimensione dell'embedding
    vs = FAISS.from_documents(
        documents=chunks,
        embedding=embeddings
    )

    Path(persist_dir).mkdir(parents=True, exist_ok=True)
    vs.save_local(persist_dir)
    return vs


def load_or_build_vectorstore(settings: Settings, embeddings: HuggingFaceEmbeddings, docs: List[Document]) -> FAISS:
    """
    Tenta il load di un indice FAISS persistente; se non esiste, lo costruisce e lo salva.
    """
    persist_path = Path(settings.persist_dir)
    index_file = persist_path / "index.faiss"
    meta_file = persist_path / "index.pkl"

    if index_file.exists() and meta_file.exists():
        # Dal 2024/2025 molte build richiedono il flag 'allow_dangerous_deserialization' per caricare pkl locali
        return FAISS.load_local(
            settings.persist_dir,
            embeddings,
            allow_dangerous_deserialization=True
        )

    chunks = split_documents(docs, settings)
    return build_faiss_vectorstore(chunks, embeddings, settings.persist_dir)


def make_retriever(vector_store: FAISS, settings: Settings):
    """
    Configura il retriever. Con 'mmr' otteniamo risultati meno ridondanti e più coprenti.
    """
    if settings.search_type == "mmr":
        return vector_store.as_retriever(
            search_type="mmr",
            search_kwargs={"k": settings.k, "fetch_k": settings.fetch_k, "lambda_mult": settings.mmr_lambda},
        )
    else:
        return vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": settings.k},
        )


def format_docs_for_prompt(docs: List[Document]) -> str:
    """
    Prepara il contesto per il prompt, includendo citazioni [source].
    """
    lines = []
    for i, d in enumerate(docs, start=1):
        src = d.metadata.get("source", f"doc{i}")
        lines.append(f"[source:{src}] {d.page_content}")
    return "\n\n".join(lines)


def build_rag_chain(llm, retriever):
    """
    Costruisce la catena RAG (retrieval -> prompt -> LLM) con citazioni e regole anti-hallucination.
    """
    system_prompt = (
        "Sei un assistente esperto. Rispondi in italiano. "
        "Usa esclusivamente il CONTENUTO fornito nel contesto. "
        "Se l'informazione non è presente, dichiara che non è disponibile. "
        "Includi citazioni tra parentesi quadre nel formato [source:...]. "
        "Sii conciso, accurato e tecnicamente corretto."
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human",
         "Domanda:\n{question}\n\n"
         "Contesto (estratti selezionati):\n{context}\n\n"
         "Istruzioni:\n"
         "1) Rispondi solo con informazioni contenute nel contesto.\n"
         "2) Cita sempre le fonti pertinenti nel formato [source:FILE].\n"
         "3) Se la risposta non è nel contesto, scrivi: 'Non è presente nel contesto fornito.'")
    ])

    # LCEL: dict -> prompt -> llm -> parser
    chain = (
        {
            "context": retriever | format_docs_for_prompt,
            "question": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain


def rag_answer(question: str, chain) -> str:
    """
    Esegue la catena RAG per una singola domanda.
    """
    return chain.invoke(question)


# =========================
# Esecuzione dimostrativa
# =========================

def main():
    settings = SETTINGS

    # 1) Componenti
    embeddings = get_embeddings(settings)
    llm = get_llm_from_lmstudio(settings)

    # 2) Dati simulati e indicizzazione (load or build)
    docs = simulate_corpus()
    vector_store = load_or_build_vectorstore(settings, embeddings, docs)

    # 3) Retriever ottimizzato
    retriever = make_retriever(vector_store, settings)

    # 4) Catena RAG
    chain = build_rag_chain(llm, retriever)

    # 5) Esempi di domande
    questions = [
        "Che cos'è una pipeline RAG e quali sono le sue fasi principali?",
        "A cosa serve FAISS e quali capacità offre?",
        "Cos'è MMR e perché è utile durante il retrieval?",
        "Quale dimensione hanno gli embedding prodotti da all-MiniLM-L6-v2?"
    ]

    for q in questions:
        print("=" * 80)
        print("Q:", q)
        print("-" * 80)
        ans = rag_answer(q, chain)
        print(ans)
        print()

if __name__ == "__main__":
    main()
```

---

## Note progettuali e scelte tecniche

1. **Embedding locale e gratuito**
   Usa `sentence-transformers/all-MiniLM-L6-v2` (dimensione 384). È veloce, senza costi e adatto a prototipi e piccoli sistemi RAG.

2. **Text splitting**
   `RecursiveCharacterTextSplitter` con `chunk_size=700` e `chunk_overlap=100` bilancia copertura e coerenza. Puoi regolare questi parametri se i tuoi documenti sono molto densi o molto eterogenei.

3. **FAISS persistente**
   `save_local()` e `load_local()` evitano di ricalcolare gli embedding e ricostruire l’indice ad ogni esecuzione. È importante per tempi di avvio rapidi in ambienti reali.

4. **Retriever MMR**
   `search_type="mmr"` con `fetch_k` e `lambda_mult` aiuta a ridurre la ridondanza e a coprire aspetti diversi dei documenti. Se preferisci i “top-k” più densi, passa a `search_type="similarity"`.

5. **LM Studio via endpoint OpenAI-compatible**
   `init_chat_model(..., model_provider="openai")` punta a `OPENAI_BASE_URL` con `OPENAI_API_KEY` di comodo. Imposta `LMSTUDIO_MODEL` al nome del modello caricato in LM Studio.

6. **Citations**
   Le fonti sono propagate nei metadati `source` dei `Document`. La funzione `format_docs_for_prompt` costruisce un contesto con citazioni `[source:<file>]`.

7. **Estensioni possibili**

   * Contextual compression (LLM chain extractor) per ridurre i chunk prima del prompt.
   * Filtri per metadati in fase di retrieval.
   * Valutazioni offline con LangSmith (tracing, debugging) se sposti il backend a un provider remoto.



---

## 1. **Search Type** (tipi di ricerca nel retriever)

Quando usi un retriever (ad esempio con FAISS o Qdrant) hai diversi modi di recuperare i documenti in risposta a una query. I più comuni in LangChain sono:

### a) **Similarity Search (default)**

* Recupera i documenti **più vicini alla query** nello spazio vettoriale.
* Funziona ordinando i risultati per distanza (o similarità) e prendendo i primi *k*.
* È semplice, veloce e di solito sufficiente.
* Problema: può restituire chunk molto simili tra loro (ridondanza).

```python
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # prendi i 4 più simili
)
```

### b) **MMR (Maximal Marginal Relevance)**

* Non prende solo i *più simili*, ma **bilancia somiglianza e diversità**.
* Funziona così:

  1. Trova un insieme più ampio di candidati (*fetch\_k*, es. 20)
  2. Seleziona i primi risultati bilanciando:

     * **Pertinenza** rispetto alla query
     * **Diversità** rispetto ai documenti già scelti
* Parametro chiave: **λ (lambda\_mult)**

  * 0 = privilegia solo la diversità
  * 1 = privilegia solo la pertinenza

```python
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 4, "fetch_k": 20, "lambda_mult": 0.3}
)
```

* Utile quando:

  * Vuoi **evitare ridondanza**
  * Hai documenti che trattano aspetti diversi della query
  * Vuoi aumentare **copertura informativa**

---

## 2. **Chunking e Overlap**

Quando hai documenti grandi (manuali, articoli, report), non puoi mandarli interamente all’LLM (per limiti di contesto). Si usano i **Text Splitter** per dividerli in porzioni più piccole (**chunk**).

### a) **Chunk Size**

* È la lunghezza massima di un pezzo di documento (in caratteri o token).
* Scelta critica:

  * Troppo piccolo → perdita di contesto, pezzi non informativi
  * Troppo grande → rischi di superare il limite del modello e di avere embedding poco precisi

Esempio:

```python
splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,   # ~700 caratteri per chunk
    chunk_overlap=100
)
```

Qui ogni chunk avrà circa **700 caratteri** (con punteggiatura come separatori preferiti).

### b) **Chunk Overlap**

* È la **sovrapposizione** tra un chunk e il successivo (in caratteri).
* Serve per **non perdere il contesto alle frontiere dei chunk**.
* Esempio: se un paragrafo è lungo 710 caratteri e il chunk size è 700 senza overlap, rischi di “tagliare” frasi a metà.
* Con overlap=100:

  * Il chunk1 contiene 0–700
  * Il chunk2 contiene 600–1300
  * Le frasi tra 600–700 appaiono in entrambi, preservando il contesto.

### c) Linee guida pratiche

* **Chunk size**:

  * 500–1000 caratteri se usi modelli piccoli (es. MiniLM)
  * 1000–2000 caratteri per GPT-4/Claude con contesti ampi

* **Overlap**:

  * 10–20% della lunghezza del chunk
  * Tipicamente 100–200 caratteri

---

## Esempio pratico combinato

```python
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Splitting robusto con chunking e overlap
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    separators=["\n\n", "\n", ". ", "? ", "! ", "; ", ": ", ", ", " ", ""]
)

chunks = splitter.split_documents(docs)

# Retriever con MMR per ridurre ridondanza
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20, "lambda_mult": 0.3}
)
```

---

## Conclusione

* **Search Type**:

  * `similarity`: prende i più simili → veloce, ma rischia ridondanza
  * `mmr`: bilancia pertinenza e diversità → più copertura, meno ridondanza

* **Chunking & Overlap**:

  * Chunk size determina la granularità delle unità di ricerca
  * Overlap evita di perdere il contesto tra due chunk adiacenti

Entrambi sono strumenti cruciali per garantire che un sistema RAG sia **accurato, completo e robusto**.

---

## Documenti reali

Per utilizzare **documenti reali** invece di testi simulati nella tua pipeline RAG, devi costruire una funzione che **carichi file da disco**, li legga, e li converta in oggetti `Document` compatibili con LangChain.

Vediamo come fare in modo semplice, ordinato e professionale.

---

## Obiettivo della funzione

Sostituire questa funzione:

```python
def simulate_corpus() -> List[Document]:
    ...
```

con una versione reale che carichi i contenuti da una cartella locale contenente file `.txt`, `.md`, `.pdf`, o altri formati supportati da LangChain.

---

## Passaggi da seguire

1. Leggere i file reali da una directory locale.
2. Usare i **Document Loaders** di LangChain per convertirli in `Document`.
3. Impostare metadati utili, come il nome del file (per le citazioni).
4. Restituire una `List[Document]` identica come formato a quella della funzione simulata.

---

## Esempio: caricamento da file `.txt` e `.md`

```python
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document
from pathlib import Path
from typing import List


def load_real_documents_from_folder(folder_path: str) -> List[Document]:
    """
    Carica documenti reali da file di testo (es. .txt, .md) all'interno di una cartella.
    Ogni file viene letto e convertito in un oggetto Document con metadato 'source'.
    """
    folder = Path(folder_path)
    documents: List[Document] = []

    if not folder.exists() or not folder.is_dir():
        raise ValueError(f"La cartella '{folder_path}' non esiste o non è una directory.")

    for file_path in folder.glob("**/*"):
        if file_path.suffix.lower() not in [".txt", ".md"]:
            continue  # ignora file non supportati

        loader = TextLoader(str(file_path), encoding="utf-8")
        docs = loader.load()

        # Aggiunge il metadato 'source' per citazioni (es. nome del file)
        for doc in docs:
            doc.metadata["source"] = file_path.name

        documents.extend(docs)

    return documents
```

---

## Utilizzo

Nel tuo codice principale, sostituisci:

```python
docs = simulate_corpus()
```

con:

```python
docs = load_real_documents_from_folder("path/alla/cartella/documenti")
```

Assicurati che la cartella contenga file `.txt` o `.md` leggibili, come:

```
/documenti/
├── langchain_overview.md
├── faiss_notes.txt
├── embeddings_info.md
```

---

## Supporto per altri formati (PDF, HTML, CSV, ecc.)

LangChain supporta altri loader:

* `PyPDFLoader` per `.pdf`
* `UnstructuredFileLoader` per `.docx`, `.pptx`, `.eml`
* `BSHTMLLoader` per file HTML
* `CSVLoader` per `.csv`

Puoi combinarli usando controlli come:

```python
if file_path.suffix == ".pdf":
    loader = PyPDFLoader(str(file_path))
elif file_path.suffix == ".html":
    loader = BSHTMLLoader(str(file_path))
...
```

---

## Considerazioni aggiuntive

* I file troppo lunghi verranno poi **spezzati in chunk** tramite lo `TextSplitter`, come già previsto nel tuo codice.
* Il metadato `"source"` è importante per le **citazioni automatiche** nei prompt RAG.
* È buona norma assicurarsi che tutti i documenti siano in UTF-8 ed evitare file binari o malformati.

---



##  Come funziona la `build_rag_chain`

```python
def build_rag_chain(llm, retriever):
```

Questa funzione crea una **catena LCEL (LangChain Expression Language)** composta da 4 step:

1. **Retriever + formatter** → recupera i documenti e li trasforma in stringa.
2. **PromptTemplate** → costruisce il prompt per l’LLM con una struttura coerente.
3. **LLM** → genera la risposta.
4. **OutputParser** → estrae solo il testo generato.

---

##  Dettaglio: `system_prompt` e `ChatPromptTemplate`

### 1. `system_prompt`

Il system prompt serve a dare istruzioni **costanti** al modello, come se fosse la sua "personalità":

```python
system_prompt = (
    "Sei un assistente esperto. Rispondi in italiano. "
    "Usa esclusivamente il CONTENUTO fornito nel contesto. "
    "Se l'informazione non è presente, dichiara che non è disponibile. "
    "Includi citazioni tra parentesi quadre nel formato [source:...]. "
    "Sii conciso, accurato e tecnicamente corretto."
)
```

Questo è **statico**, non cambia a ogni input.

---

### 2. `ChatPromptTemplate` con variabili

```python
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human",
     "Domanda:\n{question}\n\n"
     "Contesto (estratti selezionati):\n{context}\n\n"
     "Istruzioni:\n"
     "1) Rispondi solo con informazioni contenute nel contesto.\n"
     "2) Cita sempre le fonti pertinenti nel formato [source:FILE].\n"
     "3) Se la risposta non è nel contesto, scrivi: 'Non è presente nel contesto fornito.'")
])
```

Qui usi **due messaggi**:

* `("system", ...)`: per il comportamento fisso del modello.
* `("human", ...)`: per l’input dell’utente, con **variabili** `{question}` e `{context}` che verranno riempite a runtime.

---

##  Cos’è la `chain` e il passaggio dei dati

```python
chain = (
    {
        "context": retriever | format_docs_for_prompt,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)
```

### Step-by-step:

1. `question`: passa direttamente il testo con `RunnablePassthrough()`.
2. `context`: passa per il retriever e una funzione che formatta i documenti.
3. `prompt`: usa `ChatPromptTemplate` per combinare domanda e contesto.
4. `llm`: genera la risposta.
5. `StrOutputParser`: prende solo il testo, escludendo metadati.

---

##  Come si esegue con `.invoke()`

```python
def rag_answer(question: str, chain) -> str:
    return chain.invoke(question)
```

### `chain.invoke(...)`:

* Prende in input una stringa (`question`).
* La passa lungo la catena.
* Recupera il contesto, costruisce il prompt, genera risposta.
* Restituisce solo la stringa finale (grazie a `StrOutputParser()`).

---

##  Riassunto visivo

```
[User Question] 
   ↓
[Retriever]
   ↓
[Contesto] + [Question]
   ↓
[Prompt Template]
   ↓
[LLM]
   ↓
[Output Parser]
   ↓
Risposta con citazioni accurate 
```

---

